In [1]:
import pandas as pd

In [6]:
abunds_list = [[-0.1549019599857431, -4.525818200290589, -0.5528881887906408, -2.462573412450483, -2.995531274488762, -2.027772732310497, -2.6808888533559165, -2.389635659261393]]
abunds_list.append([-0.15490195998574316, -4.525818200290589, -0.552888188790641, -2.462573412450484, -2.995531274488762, -2.027772732310498, -2.680888853355917, -2.389635659261393])
abunds_list.append([-0.15499321581605932, -4.543932823454733, -0.5528391447069334, -2.5755759444188024, -2.749602507988715, -2.028952937753346, -2.7146670685398893, -3.101814180614233])
abunds_list.append([-32.86547777631699, -73.38653113036969, -0.008965265191413775, -3.0000593413680834, -1.930504062808034, -3.6355356814338395, -2.714667068665365, -3.1018141792678073])
abunds = pd.DataFrame(abunds_list,columns=['h1', 'he3', 'he4', 'c12', 'n14', 'o16', 'ne20', 'mg24'])
abunds

,h1,he3,he4,c12,n14,o16,ne20,mg24
0,-0.154902,-4.525818,-0.552888,-2.462573,-2.995531,-2.027773,-2.680889,-2.389636
1,-0.154902,-4.525818,-0.552888,-2.462573,-2.995531,-2.027773,-2.680889,-2.389636
2,-0.154993,-4.543933,-0.552839,-2.575576,-2.749603,-2.028953,-2.714667,-3.101814
3,-32.865478,-73.386531,-0.008965,-3.000059,-1.930504,-3.635536,-2.714667,-3.101814


In [ ]:
import mesa_reader as mr
import numpy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

data_p3_20m = mr.MesaData('20_Solar_mass_giant/profile3.data')
data_p4_20m = mr.MesaData('20_Solar_mass_giant/profile4.data')


element_names_log = ['log_h1', 'log_he3', 'log_he4', 'log_c12', 'log_n14', 'log_o16', 'log_ne20', 'log_mg24']
element_names = ['h1', 'he3', 'he4', 'c12', 'n14', 'o16', 'ne20', 'mg24']
element_colours_dict = {'h1': 'blue', 'he3': 'yellow', 'he4': 'green', 'c12': 'red', 'n14': 'purple', 'o16': 'violet',
                        'ne20': 'magenta', 'mg24': 'grey'}
element_colours_dict_log = {'log_h1': 'blue', 'log_he3': 'yellow', 'log_he4': 'green', 'log_c12': 'red',
                            'log_n14': 'purple', 'log_o16': 'violet', 'log_ne20': 'magenta', 'log_mg24': 'grey'}
colours = ['blue', 'yellow', 'green', 'red', 'purple', 'violet', 'magenta', 'grey']

ax1 = plt.subplot2grid((4, 4), (0, 0))
for element in element_names:
    new_variable = getattr(data_p3_20m, element)
    ax1.plot(data_p3_20m.zone, new_variable, label=element, color=element_colours_dict[element])
ax1.invert_xaxis()
ax1.set_title('Model 400 $20 M_{\odot}$')
ax1.legend(loc='lower right', fontsize=5)
ax1.set_xlim(data_p3_20m.zone[-1], data_p3_20m.zone[0])

ax2 = plt.subplot2grid((4, 4), (0, 1))
for element in element_names:
    new_variable = getattr(data_p4_20m, element)
    ax2.plot(data_p4_20m.zone, new_variable, label=element, color=element_colours_dict[element])
ax2.invert_xaxis()
ax2.set_title('Model 600 $20 M_{\odot}$')
ax2.set_xlim(data_p4_20m.zone[-1], data_p4_20m.zone[0])

ax3 = plt.subplot2grid((4, 4), (0, 2))
temp = element_names_log.copy()
temp.remove('log_h1')
temp.remove('log_he3')
for element in temp:
    new_variable = getattr(data_p3_20m, element)
    ax3.plot(data_p3_20m.zone, new_variable, label=element, color=element_colours_dict_log[element])
ax3.invert_xaxis()
# ax3.legend(loc= 'lower right', fontsize = 5)
ax3.set_title('log Model 400 $20 M_{\odot}$')
ax3.set_xlim(data_p3_20m.zone[-1], data_p3_20m.zone[0])

ax4 = plt.subplot2grid((4, 4), (0, 3))
temp = element_names_log.copy()
temp.remove('log_h1')
temp.remove('log_he3')
for element in temp:
    new_variable = getattr(data_p4_20m, element)
    ax4.plot(data_p4_20m.zone, new_variable, label=element, color=element_colours_dict_log[element])
ax4.invert_xaxis()
# ax4.legend(loc= 'lower right', fontsize = 5)
ax4.set_title('log Model 600 $20 M_{\odot}$')
ax4.set_xlim(data_p4_20m.zone[-1], data_p4_20m.zone[0])


# surface abundance plot
# variables are zone tau, zone fractional abundance of element, zone mass
# surface is tau = 2/3 and less
# find zones within this tau, use zone mass and abundance to calculate mass of element in zone, sum over all zones that
# are within surface region to find net abundance in surface region, net surface element mass/net surface mass for each
# element
# plot these abundances as pie chart

# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# langs = ['C', 'C++', 'Java', 'Python', 'PHP']
# students = [23,17,35,29,12]
# ax.bar(langs,students)
# plt.show()

ax5 = plt.subplot2grid((4, 4), (1, 0), colspan=2)
# H surface abundance
surf_abundances = []
for element in element_names_log:
    i = 1
    element_mass_in_zone = 0
    net_mass = 0
    new_variable = getattr(data_p3_20m, element)

    while data_p3_20m.tau[i] <= 7/10:
        element_mass_in_zone = element_mass_in_zone + new_variable[i] * data_p3_20m.mass[i]
        net_mass = net_mass + data_p3_20m.mass[i]

        i = i + 1

    surf_abundances.append(element_mass_in_zone/net_mass)
print(element_names, surf_abundances)
ax5.bar(element_names, surf_abundances, color = colours)
ax5.set_title('log $20 M_{\odot}$ Surface Abundance Model 400 ' + r'$\tau \leq \frac{7}{10}$')


ax6 = plt.subplot2grid((4, 4), (1, 2), colspan=2)
# H surface abundance
surf_abundances = []
for element in element_names_log:
    i = 1
    element_mass_in_zone = 0
    net_mass = 0
    new_variable = getattr(data_p4_20m, element)
    while data_p4_20m.tau[i] <= 7/10:
        element_mass_in_zone = element_mass_in_zone + new_variable[i] * data_p4_20m.mass[i]
        net_mass = net_mass + data_p4_20m.mass[i]
        i = i + 1

    surf_abundances.append(element_mass_in_zone/net_mass)
print(element_names, surf_abundances)
ax6.bar(element_names, surf_abundances, color = colours)
ax6.set_title('log $20 M_{\odot}$ Surface Abundance Model 600 ' + r'$\tau \leq \frac{7}{10}$')


#####################################################################################
# O NE WD STARTS
#####################################################################################


data_p2_wd = mr.MesaData('make_o_ne_wd/profile2.data')
data_p9_wd = mr.MesaData('make_o_ne_wd/profile9.data')


ax7 = plt.subplot2grid((4, 4), (2, 0))
for element in element_names:
    new_variable = getattr(data_p2_wd, element)
    ax7.plot(data_p2_wd.zone, new_variable, label=element, color=element_colours_dict[element])
ax7.invert_xaxis()
ax7.set_title('Model 400 WD')
ax7.legend(loc='lower right', fontsize=5)
ax7.set_xlim(data_p2_wd.zone[-1], data_p2_wd.zone[0])

ax8 = plt.subplot2grid((4, 4), (2, 1))
for element in element_names:
    new_variable = getattr(data_p9_wd, element)
    ax8.plot(data_p9_wd.zone, new_variable, label=element, color=element_colours_dict[element])
ax8.invert_xaxis()
ax8.set_title('Model 1600 WD')
ax8.set_xlim(data_p9_wd.zone[-1], data_p9_wd.zone[0])

ax9 = plt.subplot2grid((4, 4), (2, 2))
temp = element_names_log.copy()
temp.remove('log_h1')
temp.remove('log_he3')
for element in temp:
    new_variable = getattr(data_p2_wd, element)
    ax9.plot(data_p2_wd.zone, new_variable, label=element, color=element_colours_dict_log[element])
ax9.invert_xaxis()
# ax3.legend(loc= 'lower right', fontsize = 5)
ax9.set_title('log Model 400 WD')
ax9.set_xlim(data_p2_wd.zone[-1], data_p2_wd.zone[0])

ax10 = plt.subplot2grid((4, 4), (2, 3))
temp = element_names_log.copy()
temp.remove('log_h1')
temp.remove('log_he3')
for element in temp:
    new_variable = getattr(data_p9_wd, element)
    ax10.plot(data_p9_wd.zone, new_variable, label=element, color=element_colours_dict_log[element])
ax10.invert_xaxis()
# ax4.legend(loc= 'lower right', fontsize = 5)
ax10.set_title('log Model 1600 WD')
ax10.set_xlim(data_p9_wd.zone[-1], data_p9_wd.zone[0])


# surface abundance plot
# variables are zone tau, zone fractional abundance of element, zone mass
# surface is tau = 2/3 and less
# find zones within this tau, use zone mass and abundance to calculate mass of element in zone, sum over all zones that
# are within surface region to find net abundance in surface region, net surface element mass/net surface mass for each
# element
# plot these abundances as pie chart

# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# langs = ['C', 'C++', 'Java', 'Python', 'PHP']
# students = [23,17,35,29,12]
# ax.bar(langs,students)
# plt.show()

ax11 = plt.subplot2grid((4, 4), (3, 0), colspan=2)
# H surface abundance
surf_abundances = []
for element in element_names_log:
    i = 1
    element_mass_in_zone = 0
    net_mass = 0
    new_variable = getattr(data_p2_wd, element)

    while data_p2_wd.tau[i] <= 7/10:
        element_mass_in_zone = element_mass_in_zone + new_variable[i] * data_p2_wd.mass[i]
        net_mass = net_mass + data_p2_wd.mass[i]

        i = i + 1

    surf_abundances.append(element_mass_in_zone/net_mass)

ax11.bar(element_names, surf_abundances, color = colours)
ax11.set_title('log WD Surface Abundance Model 400 ' + r'$\tau \leq \frac{7}{10}$')

print(element_names, surf_abundances)

ax12 = plt.subplot2grid((4, 4), (3, 2), colspan=2)
# H surface abundance
surf_abundances = []
for element in element_names_log:
    i = 1
    element_mass_in_zone = 0
    net_mass = 0
    new_variable = getattr(data_p9_wd, element)

    while data_p9_wd.tau[i] <= 8/10:
        element_mass_in_zone = element_mass_in_zone + new_variable[i] * data_p9_wd.mass[i]
        net_mass = net_mass + data_p9_wd.mass[i]

        i = i + 1

    surf_abundances.append(element_mass_in_zone/net_mass)
print(element_names, surf_abundances)
ax12.bar(element_names, surf_abundances, color = colours)
ax12.set_title('log WD Surface Abundance Model 1600 ' + r'$\tau \leq \frac{8}{10}$')


plt.subplots_adjust(hspace=0.5)
plt.suptitle('$20 M_{\odot}$ Star and Make_O_Ne_WD, Radial and Surface Abundances with Variable ' + r'$\tau$')
plt.show()
